# Align Dbit-seq transcriptome data and images using Squidpy tools

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

# load gui for napari
#%gui qt

In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import dbitx_funcs as db
import scanpy as sc
import napari
import cv2
#import json
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path

Import issue with loess function detected. Probably due to Windows. Error ignored.
Package 'bbknn' not installed here. Error ignored.


### Read parameters

In [3]:
# read parameters file
settings_file = "../CountsToAnndata/alignment_parameters.csv"
lines = open(settings_file).readlines()
param_start = [i for i, line in enumerate(lines) if line.startswith(">parameters")][0]
dir_start = [i for i, line in enumerate(lines) if line.startswith(">directories")][0]

# read settings file
settings = pd.read_csv(settings_file, header=None)

# read parameters
parameters = pd.read_csv(settings_file, 
                       nrows=dir_start-1).dropna(how='all', axis=0).dropna(how='all', axis=1).set_index('category')
# read directories
directories = pd.read_csv(settings_file, 
                          skiprows=dir_start, usecols=range(1,6)).dropna(how='all', axis=0)
# extract parameters
channel_names = parameters.loc["channel_names", "value"].split(" ")
channel_labels = parameters.loc["channel_labels", "value"].split(" ")

# determine alignment channel
alignment_channel = [elem for elem in channel_names if "*" in elem][0]


### Check parameters

In [4]:
### Check if all necessary parameters are in the file
param_cats = ["channel_names", "channel_labels", "n_channels"]
dir_cats = ["input_transcriptome", "input_images", "output", "vertices_x", "vertices_y"]

assert np.all([elem in parameters.index for elem in param_cats]), \
    "Not all required categories found in parameter section {}".format(param_cats)
assert np.all([elem in directories.columns for elem in dir_cats]), \
    "Not all required column headers found in directory section {}".format(dir_cats)

# check if channel names and labels have same length
assert len(channel_names) == len(channel_labels), \
    "Numbers of channel_names and channel_labels differ."

# check if all input images and matrix files exist
try:
    assert np.all([os.path.isfile(f) for f in directories["input_transcriptome"]]), \
        "Not all input files exist."
except AssertionError as e:
    # check which files are missing
    missing_files = [f for f in directories["input_transcriptome"] if not os.path.isfile(f)]
    print("{} Following files are missing: {}".format(e, missing_files))
    #sys.exit()
    exit()

# check if all output names are unique
assert len(np.unique(directories["output"])) == len(directories["output"]), \
    "Output files are not unique. This would cause that one file is overwritten by another."


### Do procedure for first dataset

In [5]:
i = 0

n_datasets = len(directories)

vertices_list = [None]*n_datasets

dirs = directories.loc[i, :]

In [6]:
# get parameters for this dataset
matrix_file = dirs["input_transcriptome"]
image_dirs = glob(dirs["input_images"])
output_file = dirs["output"]
output_dir = os.path.dirname(output_file)

# check if the vertices are given in the settings file
vertices_not_given = pd.isnull(directories.loc[i, "vertices_x"]) or pd.isnull(directories.loc[i, "vertices_y"])

# check if number of images matches number of channel names
assert len(image_dirs) == len(channel_names), "Number of detected images does not match number of channel names in parameters file."

# detect get alignment marker image
alignimg_dir = [d for d in image_dirs if alignment_channel.strip("*") in d][0]

In [7]:
# create and save adata objects for this dataset
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [8]:
# read alignment image
alignment_image = cv2.imread(alignimg_dir)

In [14]:
vertices_not_given = False

In [15]:
if vertices_not_given:
    ### Select corner spots in alignment image using napari viewer
    # with napari.gui_qt():
    # https://napari.org/guides/stable/event_loop.html#intro-to-event-loop
    viewer = napari.view_image(alignment_image, 
        title="Select corner spots in alignment image {} of {} ".format(i+1, n_datasets))

In [16]:
if vertices_not_given:
    # fetch vertices (center points at cross points of alignment channels)
    corner_spots_center = viewer.layers["Points"].data.astype(int)

    # collect corner spot
    vertices_list[i] = corner_spots_center

    # save information about vertices in settings file
    # save y coordinates (row coordinates)
    settings.loc[dir_start+i+1, 5] = " ".join([str(elem[0]) for elem in vertices_list[i]])
    # save x coordinates (column coordinates)
    settings.loc[dir_start+i+1, 4] = " ".join([str(elem[1]) for elem in vertices_list[i]])
    # save settings file with coordinates of vertices
    settings.to_csv(settings_file, index=None, header=None)
else:
    # extract coordinates from directory input
    xs = [int(elem) for elem in directories.loc[i, "vertices_x"].split(" ")]
    ys = [int(elem) for elem in directories.loc[i, "vertices_y"].split(" ")]

    # add extracted coordinates to list of vertices
    vertices_list[i] = np.array([[a, b] for a, b in zip(ys, xs)])

In [19]:
images = [cv2.imread(d, -1) for d in image_dirs]

In [20]:
db.dbitseq_to_squidpy(matrix_path=matrix_file, images=images,
    resolution=int(parameters.loc["spot_width"]), 
    n_channels=int(parameters.loc["n_channels"]), 
    frame=int(parameters.loc["frame"]),
    dbitx=False, labels=channel_labels, vertices=vertices_list[i], 
    savepath=output_file)


Create adata object in 'Dbit-seq' mode
Read transcriptome matrix from N:\01 HPC\03 Team Meier\08_Projects\37_Spatial_Barcoding\37_30\data\raw_matrices\wells\A1\DGE_matrix_with_introns_min100.txt.gz...
Compute coordinates...
14x4
Align and create image metadata...
     Align images...
[ 1560 10565 10176  1175]
[ 1560 10565 10176  1175]
[ 1560 10565 10176  1175]
[ 1560 10565 10176  1175]
     Create metadata...
     Summarized aligned images and metadata.
Adata object generated.
Saving adata object...
Adata object saved into N:\01 HPC\03 Team Meier\08_Projects\37_Spatial_Barcoding\37_30\data\anndata\37_30_adata_raw_with_images_A1.h5ad
Finished


In [71]:
# input parameters
well_id = 0
alignment_id = 1
image_names = ['bf', 'align', 'phalloidin', 'dapi']

# image path
#image_path = r"N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\images"
image_path = "/Users/Johannes/Nextcloud/DbitX/data/37_30/images/"

# transcriptome path
matrix_file = r"N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\transcriptome\wells\A1\DGE_matrix_with_introns_min100.txt.gz"

# output directory
output_name = "37_28_adata_raw_with_images.h5ad"

In [66]:
# get image names
well_names = os.listdir(image_path)
well_name = well_names[well_id]
image_names = os.listdir(os.path.join(image_path,well_name))

In [67]:
# load images
images = [cv2.imread(os.path.join(image_path, well_name, name), -1) for name in image_names]

In [68]:
alignment_image = images[alignment_id]

### Select corner spots in alignment image using napari viewer

In [74]:
viewer = napari.view_image(alignment_image, title="Select corner spots in alignment image of well " + well_name)

#### Extract pivot spot and calibration points from viewer

In [70]:
# fetch vertices (center points of )
corner_spots_center = viewer.layers["Points"].data.astype(int)

In [26]:
# values from 37_28
#corner_spots_center = np.array([[ 486,  990],[ 891, 8662], [8567, 8324],[8164,  646]])

### Create adata with images and metadata

In [73]:
savepath = os.path.join(well_path, output_name)

adata = db.dbitseq_to_squidpy(matrix_path=matrix_file, images=images, labels=image_names, vertices=corner_spots_center, 
                              resolution=50, n_channels=38, frame=100, savepath=savepath)

Read transcriptome matrix...
Align and create image metadata...
     Align images...
     Create metadata...
     Summarized aligned images and metadata...
Compute coordinates...
Adata object generated.
Saving image...
Adata object saved into N:\01 HPC\03 Team Meier\10_Resources\08_Johannes Wirth\Nextcloud\DbitX\data\37_30\images\A1\37_28_adata_raw_with_images.h5ad
Finished


## Write total data to file

In [43]:
adata.write(os.path.join(well_path, output_name))